# Retail Data Analysis with Python #

## Task 1: Load the Data

In [ ]:
!source .venv/bin/activate

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import glob 
import os
from datetime import datetime

In [ ]:
# %reload_ext jupyter_ai

In [ ]:
# # NOTE: Replace 'PROVIDER_API_KEY' with the credential key's name,
# # and enter the API key when prompted by using the code shown below.

# import getpass

# # Enter your key
# token = getpass.getpass('Enter your PROVIDER API key: ')

# # Set the environment variable without displaying the full key
# os.environ['OPENAI_API_KEY'] = token

In [ ]:
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.reset_option('^display.', silent=True)

In [ ]:
# df = pd.read_excel("Online Retail.xlsx")
filepath = './projects/sales/sales/'
all_files = glob.glob(os.path.join(filepath, "*.csv"))

df = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)

In [ ]:
df.head()

In [ ]:
df.columns = df.columns.str.title()

In [ ]:
df.columns = df.columns.str.replace(" ", "", regex=True)

In [ ]:
df = df.rename(columns={'ProductVariantSkuAtTimeOfSale':'SKU'})

In [ ]:
df.columns

In [ ]:
df['BillingRegion'] = np.where(
    (df['BillingRegion'].isna()) & (df['SalesChannel'] == 'Point of Sale'),
    'Colorado',
    df['BillingRegion']
)

In [ ]:
# df1 = df.groupby('OrderId').agg({'NetSales': 'sum', 'ShippingCharges': 'sum'}).reset_index()

In [ ]:
# %%ai chatgpt 
# I need to join df and df1 on OrderId

In [ ]:
# df_merge = df.merge(df1, on='OrderId', how='inner')

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
# Find the rows where 'InvoiceNo' contains "C" in their values
# df[df['InvoiceNo'].str.contains('C', na=False)]

In [ ]:
print("Number of missing values per column:")
print(df.isnull().sum())

print("---------------------------------------------------------------------------------------------")

print("Number of unique values per row:")
print(df.nunique())

In [ ]:
# Replace null CustomerIDs with "Unknown"
# df['CustomerId'].fillna("Unknown", inplace=True)
df.fillna({"CustomerId": "Unknown"}, inplace=True)
# Filter rows with non-null 'Description'
df = df[df['ProductTitle'].notna()]

In [ ]:
columns_to_drop = ["OrderId", "SKU"]
df.drop(columns=columns_to_drop, inplace=True)

In [ ]:
df.head(10)

In [ ]:
df = df.drop_duplicates()
df.info()

In [ ]:
# Calculate the gross amount for each transaction (Quantity * UnitPrice)
df = df.assign(Gross=df['QuantityOrdered'] * df['CostOfGoodsSold'])
df

In [ ]:
df.describe()

In [ ]:
# Calculate measures of central tendency
median = df.median(numeric_only=True)
median

In [ ]:
df['Day'] = df['Day'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))

In [ ]:
# Create a new column 'MonthYear' with the month and year only for Data Visualization
df['MonthYear'] = df['Day'].dt.to_period('M').copy()

# Print the DataFrame with the 'MonthYear' column
df

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Group the data by 'MonthYear' and calculate the gross purchase for each month
monthly_data = df.groupby('MonthYear')['Gross'].sum().reset_index()

# Convert the 'MonthYear' column to string format
monthly_data['MonthYear'] = monthly_data['MonthYear'].astype(str)

# Set the figure size using subplots
fig, ax = plt.subplots(figsize=(10, 5))

# Automatic formatting to the x-axis labels when dealing with dates
fig.autofmt_xdate()

# Create a colorful plot with markers and lines
plt.plot(monthly_data['MonthYear'], monthly_data['Gross'], marker='o', linestyle='-', color='dodgerblue', label='Gross')

# Customize the plot
plt.title('Gross by Month', fontsize=16)
plt.xlabel('MonthYear', fontsize=12)
plt.ylabel('Gross', fontsize=12)
plt.xticks(rotation=45, fontsize=10)
plt.yticks(fontsize=10)
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend(loc='upper right', fontsize=12)

# Add a background color to the plot
ax.set_facecolor('lightgray')

# Add more colors and styles if needed
# Example: ax.spines['top'].set_color('none')

plt.show()  # Show the colorful plot

In [ ]:
# Create a copy of the 'Month' column
df['Month'] = df['Day'].dt.month.values

# Create a copy of the 'DayOfWeek' column
df['DayOfWeek'] = df['Day'].dt.day_name().values

# Calculate gross sales for each month
monthly_sales = df.groupby('Month')['Gross'].sum()

# Calculate gross sales for each day of the week
daily_sales = df.groupby('DayOfWeek')['Gross'].sum()

# Identify the busiest month
busiest_month = monthly_sales.idxmax()

# Identify the busiest day of the week
busiest_day = daily_sales.idxmax()

# Print the results
print("Busiest Month (in terms of sales):", busiest_month)
print("Busiest Day of the Week (in terms of sales):", busiest_day)

In [ ]:
import matplotlib.pyplot as plt

# Create a figure and set its size
plt.figure(figsize=(10, 6))

# Define the colors for the bars
colors = ['royalblue', 'forestgreen', 'dodgerblue', 'mediumseagreen', 'lightcoral', 'lightseagreen', 'orange']

# Create a bar plot
plt.bar(daily_sales.index, daily_sales.values, color=colors)

# Set labels and title
plt.xlabel('Day of the Week', fontsize=12)
plt.ylabel('Gross Amount', fontsize=12)
plt.title('Sales Trend by Day of the Week', fontsize=16)

# Set custom x-axis labels
days_of_week = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
plt.xticks(range(len(days_of_week)), days_of_week, rotation=45, fontsize=10)

# Add gridlines
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Show the plot
plt.show()

In [ ]:
# Calculate the gross purchase amount for each customer
customer_total_purchase = df.groupby('CustomerId')['Gross'].sum()

# Sort the customers based on the gross purchase amount in descending order
most_valuable_customers = customer_total_purchase.sort_values(ascending=False)

# Convert the Series to a DataFrame
most_valuable_customers_df = pd.DataFrame(most_valuable_customers, columns=['Gross']).reset_index()

# Count the total number of customers in 'df'
total_customers = df['CustomerId'].nunique()

# Print the gross number of customers and the top 10 customers
print('\033[1m' + f"Among {total_customers} customers, the top 10 customers are:" +'\033[0m', list(most_valuable_customers_df['CustomerId'][:10]))

In [ ]:
# Find most valuable items in the list
items_total_sell = df.groupby('ProductTitle')['Gross'].sum()

# Sort the items based on the total sells in descending order
most_valuable_items = items_total_sell.sort_values(ascending=False)

# Convert the Series to a DataFrame
most_valuable_items_df = pd.DataFrame(most_valuable_items, columns=['Gross']).reset_index()

# Count the total number of items in 'df'
total_items = df['ProductTitle'].nunique()

# Print the total number of items and the top 10 items
print('\033[1m' + f"Among {total_items} items, the 10 most valuable items are:" +'\033[0m', list(most_valuable_items_df['ProductTitle'][:10]))

In [ ]:
# Find the country with the most sales in the list
statewise_sell = df.groupby('BillingRegion')['Gross'].sum()

# Sort countries based on the total sales in descending order
top_selling_state = statewise_sell.sort_values(ascending=False)

# Convert the Series to a DataFrame
top_selling_state_df = pd.DataFrame(top_selling_state, columns=['Gross']).reset_index()

# Count the total number of unique countries in 'df'
total_states = df['BillingRegion'].nunique()

# Print the total number of countries and the top 10 selling countries
print('\033[1m' + f"Among {total_states} states, the top 10 selling states are:" +'\033[0m', list(top_selling_state_df['BillingRegion'][:10]))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Create a figure and set its size
plt.figure(figsize=(8, 6))

# Create a box plot for the 'Gross' column
sns.boxplot(data=df, y='Gross', color='lightblue', showfliers=False)  # showfliers=False to remove outliers

# Add a title and labels
plt.title('Distribution of Gross Amount', fontsize=16)
plt.xlabel('Gross', fontsize=12)

# Show the plot
plt.show()

In [ ]:
from scipy import stats

# Calculate the Z-Score for the 'Quantity' column
z_scores = stats.zscore(df['Gross'])

# Create a boolean mask to identify outliers (both positive and negative Z-scores)
threshold = 3
outlier_mask = abs(z_scores) > threshold

# Extract outliers from the DataFrame
outliers = df[outlier_mask]

outliers.shape

In [ ]:
# Remove outliers from the main dataset
without_outliers_online_retail = df[~outlier_mask]

# Statistical Summary of data without outliers
summary_without_outliers = without_outliers_online_retail.describe()
summary_without_outliers

In [ ]:
df.describe() # Statistical Summary of data with outliers

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Create a figure and set its size
plt.figure(figsize=(8, 6))

# Customize the color palette
# colors = sns.color_palette("Paired")  # You can choose a different palette

# Create a box plot for the 'Gross' column without outliers
sns.boxplot(data=without_outliers_online_retail, y='Gross', width=0.5, linewidth=2)

# Add a title and labels
plt.title('Distribution of Gross Amount (Without Outliers)', fontsize=16)
plt.xlabel('Gross', fontsize=12)
plt.ylabel('Gross Amount', fontsize=12)

# Add a grid to the plot
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Show the plot
plt.show()